In [1]:
########################  Start here  ####################################
from IPython.display import display, HTML; display(HTML("<style>.container { width:100% !important; }</style>"))
import threading;import time;import pynq.lib.rgbled as rgbled;from pynq.overlays.base import BaseOverlay;import asyncio
import socket;import subprocess;import platform
base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODB
#include "gpio.h"
#include "pyprintf.h"
void write_gpioB(unsigned int pin, unsigned int val){//Function to turn on/off a selected pin of PMODX
    if (val > 1){pyprintf("pin value must be 0 or 1");}
    gpio pin_out = gpio_open(pin);gpio_set_direction(pin_out, GPIO_OUT);gpio_write(pin_out, val);
}
unsigned int read_gpioB(unsigned int pin){//Function to read the value of a selected pin of PMODB
    gpio pin_in = gpio_open(pin);gpio_set_direction(pin_in, GPIO_IN);return gpio_read(pin_in);
}

In [3]:
%%microblaze base.PMODA
#include "gpio.h"
#include "pyprintf.h"
void write_gpioA(unsigned int pin, unsigned int val){//Function to turn on/off a selected pin of PMODX
    if (val > 1){pyprintf("pin value must be 0 or 1");}
    gpio pin_out = gpio_open(pin);gpio_set_direction(pin_out, GPIO_OUT);gpio_write(pin_out, val);
}
unsigned int read_gpioA(unsigned int pin){//Function to read the value of a selected pin of PMODB
    gpio pin_in = gpio_open(pin);gpio_set_direction(pin_in, GPIO_IN);return gpio_read(pin_in);
}

In [4]:
## Multiprocessing, sharing information between processes, this is simple form
import multiprocessing, os;from multiprocessing import Process, Lock; 
from multiprocessing.sharedctypes import Value, Array
from ctypes import Structure, c_double;import ctypes;from array import *
### CONSTANTS ###
SERVER, LOW, HIGH = 0, 0, 1;TONE_FREQ1, TONE_FREQ2 = 400, 100
DEBUG, OPC = True, 1;ETH_ENDL = "\r\n";Parser_Thr_flag = True 
threads_all_flag, Parser_Thr_flag, btn0_flag, btn1_flag, btn2_flag, btn3_flag = True, True, False, False, False, False    # "cond" will keep threads live or kill them all
threads = [];arg1 = arg2 = ""
procs = []
def buzzer():
    global SERVER
    tone_freq = TONE_FREQ1 if SERVER == 0 else TONE_FREQ2
    start = time.time();print(start)
    for i in range (1000):
        if SERVER == 0:
            write_gpioA(0, LOW)
        else:
            write_gpioB(0, LOW)
        time.sleep(1/(2*tone_freq))
        if SERVER == 0:
            write_gpioA(0, HIGH)
        else:
            write_gpioB(0, HIGH)
        time.sleep(1/(2*tone_freq))
        checktime = time.time()
        if checktime-start > 0.5:break# just 0.5 sec duration
    return 1
def worker_thr_parser():#[_l]: threading lock (resource), [num]: index representing the LED and thread number.
    global cmd,arg1,arg2,DEBUG,OPC, threads_all_flag, Parser_Thr_flag, dac, adc, leds, btn0_flag, btn1_flag, btn2_flag, btn3_flag, State
    while threads_all_flag:
        try:
            global thr_adcwf,Parser_Thr_flag
            s= socket.socket(socket.AF_INET,socket.SOCK_STREAM)
            s.bind(('',4944)) #bind to all ip in the range
            s.listen(1) #listen for 1 connection
            Parser_Thr_flag = True
            while Parser_Thr_flag:
                try:
                    connection, client_address = s.accept();print ("connection from :" +str(client_address))
                except Exception as inst:
                    s.shutdown(socket.SHUT_RDWR)
                    s.close()
                    Parser_Thr_flag = False
                    break
                while Parser_Thr_flag:
                    try:
                        connection.send(bytes(""+ETH_ENDL, "utf-8"))#connection.send(bytes("Enter cmd :"+ETH_ENDL, "utf-8"))
                        msg = connection.recv(8192).decode();msg = msg.upper();msg=msg[:len(msg)-2];print(msg+'<')
                    except Exception as inst:
                        s.shutdown(socket.SHUT_RDWR)
                        s.close()
                        Parser_Thr_flag = False
                        break

                    msg = msg.split()
                    cmd = arg1 = arg2 = ''
                    if (len(msg)>0):cmd = msg[0]
                    if (len(msg)>1):arg1=msg[1]
                    if (len(msg)>2):arg2=msg[2]
                    if (DEBUG):print(cmd+','+arg1+','+arg2)
                    if cmd == "CMD1":
                        ret = buzzer()
                        connection.send(bytes(str(cmd+" = "+str(ret))+ETH_ENDL, 'utf-8'))                   
                    elif cmd == "EXIT" or Parser_Thr_flag == False or threads_all_flag == False:
                        connection.send(bytes("Adios"+ETH_ENDL, 'utf-8'))
                        time.sleep(1)
                        s.shutdown(socket.SHUT_RDWR)
                        Parser_Thr_flag = False
                        time.sleep(0.3)
                        #s.close()
                        break   
                    else:
                        time.sleep(0.3)
        finally:
            s.close()
        time.sleep(2)

def run_client(_btn0,_btn1,_btn2,_btn3,_sw0,_sw1):#Client B     ,sw0_state,sw1_state
    CLIENT = 'B';ETH_ENDL = "\r\n"
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_ip = "127.0.0.1"  # replace with the server's IP address
    server_port = 4944  # replace with the server's port number
    client.connect((server_ip, server_port))    
    sw_states = []
    val = _sw0.read();sw_states.append(val)
    val = _sw1.read();sw_states.append(val)
    print("Client: Initial switches states : "+str(sw_states[0])+str(sw_states[1]))
    print("Client " + CLIENT+"is connected")
    threads_all_flag = True
    while threads_all_flag:#valid commands: "cmd1" "exit"
        time.sleep(0.2)
        if (_btn1.read()==1):
            msg = "cmd1"+ETH_ENDL
            print("Client " + CLIENT+" transmitted: "+msg)
            client.send(msg.encode("utf-8")[:1024])
            time.sleep(0.2)
            response = client.recv(1024)# receive message from the server
            response = response.decode("utf-8")
            print("Response by Server: "+response.rstrip())             
            # if server sent us "closed" in the payload, we break out of the loop and close our socket
            msg = msg.upper();
            if response.lower() == "closed":
                break
        if (_sw1.read()!=sw_states[1]):
            msg = "exit"+ETH_ENDL
            print("Client " + CLIENT+" transmitted: "+msg)
            client.send(msg.encode("utf-8")[:1024])
            time.sleep(0.2)
            response = client.recv(1024)# receive message from the server
            response = response.decode("utf-8")
            print("Response by Server: "+response.rstrip())             
            # if server sent us "closed" in the payload, we break out of the loop and close our socket
            msg = msg.upper();
            if response.lower() == "closed" or msg=='EXIT'+ETH_ENDL:
                threads_all_flag = False
                break     
                
    # close client socket (connection to the server)
    client.close()
    print("Connection to server closed")

async def get_btns(_loop):
    btns = base.btns_gpio
    switches = base.switches_gpio
    global DEBUG,threads_all_flag, Parser_Thr_flag, leds, btn0_flag, btn1_flag, btn2_flag, btn3_flag, State
    clientLaunched_flag = False
    while threads_all_flag:
        await asyncio.sleep(0.1)
        if (btns[0].read()==1 and clientLaunched_flag==False):#Launch client #A
            clientLaunched_flag = True
            await asyncio.sleep(0.3)
            print("button 0 pressed")
            time.sleep(0.2)
            btn0_flag = True
            print("Launching new process Client B")
            p = multiprocessing.Process(target=run_client, args=(btns[0],btns[1],btns[2],btns[3],switches[0],switches[1]))
            p.start()
            os.system("taskset -p {}".format(p.pid)) # taskset is an os command to pin the process to a specific CPU
            procs.append(p)
            print('Process: {}, PID: {} Started'.format(p.name, p.pid))   
            time.sleep(2)

def add(_i, a, b, returnValuePtr):# _i : Index of the process, a, b : Integers to add, returnValues : Multiprocessing array in which we will store the result at index _i
    returnValuePtr[_i] = a+b 
##### Start here ######
print("User interaction(Buttons and Switches):\n   \
        0 - Server A, launches a New Process Client B\n   \
        1 - Client B, Transmits “cmd1” to Server A to emit on Buzzer A \n   \
        2 - Server B, launches a New Process Client A\n   \
        3 - Client A, Transmits “cmd1” to Server B to emit on Buzzer B\n  \
        Switch 0–Toggle switch for  Client A, Transmit exit to Server B to CLOSE All\n  \
        Switch 1–Toggle switch for  Client B, Transmit exit to Server A to CLOSE All\n")
### Create needed Threads (don't start them up yet) 
thr1 = threading.Thread(target=worker_thr_parser, args=())
threads.append(thr1)
for thr in threads:### Start Threads  
    thr.start()

loop = asyncio.new_event_loop();  # Instance event_loop object
loop.create_task(get_btns(loop)); # take user input buttons    

loop.run_forever()
loop.close()
### Stop Live Threads, after  commanded by user buttons
for thr in threads:
    thr.join()
    print('{} joined'.format(t.name))
    


User interaction(Buttons and Switches):
           0 - Server A, launches a New Process Client B
           1 - Client B, Transmits “cmd1” to Server A to emit on Buzzer A 
           2 - Server B, launches a New Process Client A
           3 - Client A, Transmits “cmd1” to Server B to emit on Buzzer B
          Switch 0–Toggle switch for  Client A, Transmit exit to Server B to CLOSE All
          Switch 1–Toggle switch for  Client B, Transmit exit to Server A to CLOSE All

button 0 pressed
Launching new process Client B
Client: Initial switches states : 01
Client Bis connected
pid 1061's current affinity mask: 3
connection from :('127.0.0.1', 35852)
Process: Process-1, PID: 1061 Started
Client B transmitted: cmd1

Response by Server: CMD1<
CMD1,,
1667276395.1102114

Client B transmitted: cmd1

CMD1<
CMD1,,
1667276404.382376
Response by Server: CMD1 = 1
Client B transmitted: cmd1

CMD1<
CMD1,,
1667276407.4335263
Response by Server: CMD1 = 1
Client B transmitted: exit

EXIT<
EXIT,,
Respo

Exception in thread Thread-5 (worker_thr_parser):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_858/2642262845.py", line 35, in worker_thr_parser
OSError: [Errno 98] Address already in use


KeyboardInterrupt: 